In [3]:
''' 
    Joint transforms (JT) obtained by solving associated systems of ODEs
    Setting is d=2 or d=3, with possible conditioning at time t_0>0
    Includes JT of e.g. (Q(t),\lambda(t)) and of JT with multiple timepoints e.g. (Q(t),Q(t+\tau))

'''

# Solving the ODE for tilde_s
# Introducing all the relevant functions involved in the ODEs

# Input is matrix B (deterministic version)
def beta(b,s,j):
    b_j = np.transpose(b)[j]
    return np.exp(-np.dot(b_j,s))

# Known function of hat_z
def hat_z(z,mu,u,j):
    return 1 + (z[j]-1)*np.exp(-mu[j]*u)


# Define the ODE that the marginal tilde_s_j solves, starting in t=0
def tilde_s(s,u,a,b,mu,z,j):
    dsdu = a[j]*s[j] + hat_z(z,mu,u,j)*beta(b,s,j) - 1
    return -dsdu

# ODE for tilde_s_j with inital condition at time t0
def tilde_s_init(s,u,a,b,mu,z,j,t0):
    dsdu = a[j]*s[j] + hat_z(z,mu,u-t0,j)*beta(b,s,j) - 1
    return -dsdu

# Version for random marks - we choose B exponential
# Laplace transform in case each entry B_{ij}~Exp(b_{ij}) independently (!)
def beta_expLT(b,s,j):
    return np.prod(np.array(1/np.transpose(b)[j])/(np.array(s)+np.array(1/np.transpose(b)[j])))

# Define the ODE that the marginal tilde_s_j solves, starting in t=0, with random marks
def tilde_s_rm(s,u,a,b,mu,z,j):
    dsdu = a[j]*s[j] + hat_z(z,mu,u,j)*beta_expLT(b,s,j) - 1
    return -dsdu

# ODE for tilde_s_j with inital condition at time t0, with random marks
def tilde_s_init_rm(s,u,a,b,mu,z,j,t0):
    dsdu = a[j]*s[j] + hat_z(z,mu,u-t0,j)*beta_expLT(b,s,j) - 1
    return -dsdu

In [16]:
''' 
    Trivariate setting
    Joint transform compuations
'''

# Define the ODE for the system of ODEs for tilde_s in d=3
def tilde_s_system_tri(s,u,a,b,mu,z):
    dsdu = [tilde_s(s,u,a,b,mu,z,0), tilde_s(s,u,a,b,mu,z,1), tilde_s(s,u,a,b,mu,z,2)]
    return dsdu

def tilde_s_system_init_tri(s,u,a,b,mu,z,t0):
    dsdu = [tilde_s_init(s,u,a,b,mu,z,0,t0), tilde_s_init(s,u,a,b,mu,z,1,t0), tilde_s_init(s,u,a,b,mu,z,2,t0)]
    return dsdu


# with random exponential marks d=3
def tilde_s_system_tri_rm(s,u,a,b,mu,z):
    dsdu = [tilde_s_rm(s,u,a,b,mu,z,0), tilde_s_rm(s,u,a,b,mu,z,1), tilde_s_rm(s,u,a,b,mu,z,2)]
    return dsdu

def tilde_s_system_init_tri_rm(s,u,a,b,mu,z,t0):
    dsdu = [tilde_s_init_rm(s,u,a,b,mu,z,0,t0), tilde_s_init_rm(s,u,a,b,mu,z,1,t0), tilde_s_init_rm(s,u,a,b,mu,z,2,t0)]
    return dsdu



''' JT conditional at t=0'''

# Joint Transform for deterministic marks
def zeta_trivariate(t,s,z):
    # Note that s and z are (3-dimensional) vectors
    t_vec_ODE = np.linspace(0,t,M)
    s_vec = integrate.odeint(tilde_s_system_tri, s, t_vec_ODE, args=(alpha_tri,B_tri,mu_tri,z))
    exp_component1 = np.exp(-l_inf_tri[0]*s_vec[-1][0] - l_inf_tri[0]*alpha_tri[0]*integrate.simps(s_vec[:,0],t_vec_ODE))
    exp_component2 = np.exp(-l_inf_tri[1]*s_vec[-1][1] - l_inf_tri[1]*alpha_tri[1]*integrate.simps(s_vec[:,1],t_vec_ODE))
    exp_component3 = np.exp(-l_inf_tri[2]*s_vec[-1][2] - l_inf_tri[2]*alpha_tri[2]*integrate.simps(s_vec[:,2],t_vec_ODE))
    return exp_component1*exp_component2*exp_component3

# Joint transform for random (exponential) marks
def zeta_trivariate_rm(t,s,z):
    t_vec_ODE = np.linspace(0,t,M)
    s_vec = integrate.odeint(tilde_s_system_tri_rm, s, t_vec_ODE, args=(alpha_tri,B_tri,mu_tri,z))
    exp_component1 = np.exp(-l_inf_tri[0]*s_vec[-1][0] - l_inf_tri[0]*alpha_tri[0]*integrate.simps(s_vec[:,0],t_vec_ODE))
    exp_component2 = np.exp(-l_inf_tri[1]*s_vec[-1][1] - l_inf_tri[1]*alpha_tri[1]*integrate.simps(s_vec[:,1],t_vec_ODE))
    exp_component3 = np.exp(-l_inf_tri[2]*s_vec[-1][2] - l_inf_tri[2]*alpha_tri[2]*integrate.simps(s_vec[:,2],t_vec_ODE))
    return exp_component1*exp_component2*exp_component3


''' 
    Finite Difference schemes 
    Some combinations chosen, rest can be derived similarly
'''

# First order moment of Q_1(t)
def derivative_JT_tri_z1(JT, t, s, z, h):
    return (JT(t,s,[z[0]+.5*h,z[1],z[2]]) - JT(t,s,[z[0]-.5*h,z[1],z[2]]))/h

# First order moment of L_1(t)
def derivative_JT_tri_s1(JT, t, s, z, h):
    return -(JT(t,[s[0]+.5*h, s[1],s[2]],z) - JT(t,[s[0]-.5*h,s[1],s[2]],z))/h

# Second order reduced moment E[Q_1(t)^{[2]}]
def derivative2nd_JT_tri_z1z1(JT, t, s, z, h = 10**(-3)):
    return (JT(t, s, [z[0]+h,z[1],z[2]]) - 2*JT(t, s, z) +JT(t, s, [z[0]-h,z[1],z[2]]))/(h**2)
     
# Second order mixed moment E[Q_1(t)*Q_2(t)]
def derivative2nd_JT_tri_z1z2(JT, t, s, z, h = 10**(-3)):
    return (JT(t, s, [z[0]+h,z[1]+h,z[2]]) - JT(t, s, [z[0]+h,z[1]-h, z[2]]) - JT(t, s, [z[0]-h,z[1]+h, z[2]]) + JT(t, s, [z[0]-h,z[1]-h,z[2]]))/(4*h**2)

# Second order mixed moment E[Q_1(t)*\lambda_1(t)]
def derivative2nd_JT_tri_z1s1(JT, t, s, z, h = 10**(-3)):
    return -(JT(t, [s[0]+h,s[1],s[2]], [z[0]+h,z[1],z[2]]) - JT(t,  [s[0]-h,s[1],s[2]], [z[0]+h,z[1],z[2]]) - JT(t,  [s[0]+h,s[1],s[2]], [z[0]-h,z[1],z[2]]) + JT(t, [s[0]-h,s[1],s[2]],[z[0]-h,z[1],z[2]]))/(4*h**2)

# Second order moment E[\lambda_1(t)^2]
def derivative2nd_JT_tri_s1s1(JT, t, s, z, h = 10**(-3)):
    return (JT(t, [s[0]+h,s[1],s[2]], z) - 2*JT(t, s, z) +JT(t, [s[0]-h,s[1],s[2]],z))/(h**2)

# Second order mixed moment E[\lambda_1(t)*\lambda_2(t)]
def derivative2nd_JT_tri_s1s2(JT, t, s, z, h = 10**(-3)):
    return (JT(t, [s[0]+h,s[1]+h,s[2]], z) - JT(t,  [s[0]+h,s[1]-h,s[2]], z) - JT(t,  [s[0]-h,s[1]+h,s[2]], z) + JT(t, [s[0]-h,s[1]-h,s[2]],z))/(4*h**2)




In [15]:

''' 
    Bivariate setting
    Joint transform compuations
'''


# Define the ODE for the system of ODEs for tilde_s in d=2
def tilde_s_system_bi(s,u,a,b,mu,z):
    dsdu = [tilde_s(s,u,a,b,mu,z,0), tilde_s(s,u,a,b,mu,z,1)]
    return dsdu

def tilde_s_system_init_bi(s,u,a,b,mu,z,t0):
    dsdu = [tilde_s_init(s,u,a,b,mu,z,0,t0), tilde_s_init(s,u,a,b,mu,z,1,t0)]
    return dsdu


# Define the ODE for the system of ODEs for tilde_s in d=2, with random (exponential) marks
def tilde_s_system_bi_rm(s,u,a,b,mu,z):
    dsdu = [tilde_s_rm(s,u,a,b,mu,z,0), tilde_s_rm(s,u,a,b,mu,z,1)]
    return dsdu

def tilde_s_system_init_bi_rm(s,u,a,b,mu,z,t0):
    dsdu = [tilde_s_init_rm(s,u,a,b,mu,z,0,t0), tilde_s_init_rm(s,u,a,b,mu,z,1,t0)]
    return dsdu


# Joint Transform for deterministic marks
def zeta_bivariate(t,s,z):
    # Note that s and z are (2-dimensional) vectors
    t_vec_ODE = np.linspace(0,t,M)
    s_vec = integrate.odeint(tilde_s_system_bi, s, t_vec_ODE, args=(alpha_bi,B_bi,mu_bi,z))
    exp_component1 = np.exp(-l_inf_bi[0]*s_vec[-1][0] - l_inf_bi[0]*alpha_bi[0]*integrate.simps(s_vec[:,0],t_vec_ODE))
    exp_component2 = np.exp(-l_inf_bi[1]*s_vec[-1][1] - l_inf_bi[1]*alpha_bi[1]*integrate.simps(s_vec[:,1],t_vec_ODE))
    return exp_component1*exp_component2

# Joint Transform for random (exponential) marks
def zeta_bivariate_rm(t,s,z):
    # Note that s and z are (2-dimensional) vectors
    t_vec_ODE = np.linspace(0,t,M)
    s_vec = integrate.odeint(tilde_s_system_bi_rm, s, t_vec_ODE, args=(alpha_bi,B_bi,mu_bi,z))
    exp_component1 = np.exp(-l_inf_bi[0]*s_vec[-1][0] - l_inf_bi[0]*alpha_bi[0]*integrate.simps(s_vec[:,0],t_vec_ODE))
    exp_component2 = np.exp(-l_inf_bi[1]*s_vec[-1][1] - l_inf_bi[1]*alpha_bi[1]*integrate.simps(s_vec[:,1],t_vec_ODE))
    return exp_component1*exp_component2


''' Finite Difference schemes '''

# First order moment of E[Q_1(t)]
def derivative_JT_bi_z1(JT, t, s, z, h):
    return (JT(t,s,[z[0]+.5*h,z[1]]) - JT(t,s,[z[0]-.5*h,z[1]]))/h
# First order moment E[\lambda_1(t)]
def derivative_JT_bi_s1(JT, t, s, z, h):
    return -(JT(t,[s[0]+.5*h, s[1]],z) - JT(t,[s[0]-.5*h,s[1]],z))/h

# Second order moment E[\lambda_1(t)^2]
def derivative2nd_JT_bi_s1s1(JT, t, s, z, h = 10**(-3)):
    return (JT(t, [s[0]+h,s[1]], z) - 2*JT(t, [s[0],s[1]], z) +JT(t, [s[0]-h,s[1]], z))/(h**2)
# Second order mixed moment E[\lambda_1(t)*\lambda_2(t)]
def derivative2nd_JT_bi_s1s2(JT, t, s, z, h = 10**(-3)):
    return (JT(t, [s[0]+h,s[1]+h], z) - JT(t, [s[0]+h,s[1]-h], z) - JT(t, [s[0]-h,s[1]+h], z) + JT(t, [s[0]-h,s[1]-h],z))/(4*h**2)


# Second order reduced moments E[Q_1(t)^{[2]}]
def derivative2nd_JT_bi_z1z1(JT, t, s, z, h = 10**(-2)):
    sec_fact_Q1 = (JT(t, s, [z[0]+h,z[1]]) - 2*JT(t, s, z) +JT(t, s, [z[0]-h,z[1]]))/(h**2)
    return sec_fact_Q1

# Second order mixed moment E[Q_1(t)*Q_2(t)]
def derivative2nd_JT_bi_z1z2(JT, t, s, z, h = 10**(-3)):
    sec_fact_Q1Q2 = (JT(t, s, [z[0]+h,z[1]+h]) - JT(t, s, [z[0]+h,z[1]-h]) - JT(t, s, [z[0]-h, z[1]+h]) + JT(t, s, [z[0]-h,z[1]-h]))/(4*h**2)
    return sec_fact_Q1Q2

# Second order moments mixed moment E[Q_1(t)*\lambda_1(t)]
def derivative2nd_JT_bi_z1s1(JT, t, s, z, h = 10**(-3)):
    sec_fact_Q1L1 = (JT(t, [s[0]+h,s[1]], [z[0]+h,z[1]]) - JT(t, [s[0]-h,s[1]], [z[0]+h,z[1]]) - JT(t, [s[0]+h,s[1]], [z[0]-h, z[1]]) + JT(t, [s[0]-h,s[1]], [z[0]-h,z[1]]))/(4*h**2)
    return -sec_fact_Q1L1

# Third order moments E[Q_1(t){[3]}}]
def derivative3rd_JT_bi_z1z1z1(JT,t, s, z, h = 10**(-1)):
    third_fact_Q1 = (-JT(t, s, [z[0]-2*h,z[1]]) + 2*JT(t, s, [z[0]-h,z[1]]) - 2*JT(t, s, [z[0]+h,z[1]]) + JT(t, s, [z[0]+2*h,z[1]]))/(2*h**3)
    return third_fact_Q1 

# Third order moment E[\lambda_1(t)^3]
def derivative3rd_JT_bi_s1s1s1(JT,t, s, z, h = 10**(-1)):
    return -(-JT(t, [s[0]-2*h,s[1]], z) + 2*JT(t, [s[0]-h,s[1]], z) - 2*JT(t, [s[0]+h,s[1]], z) + JT(t, [s[0]+2*h,s[1]],z))/(2*h**3)



In [14]:
''' 
    Joint Transform bivariate setting
    conditional on t0
'''

# Joint transform conditional on value at t0
def zeta_bivariate_cond(t,s,z,t0,l0,q0):
    t_vec_ODE = np.linspace(t0,t,M)
    s_vec = integrate.odeint(tilde_s_system_init_bi_rm, s, t_vec_ODE, args=(alpha_bi,B_bi,mu_bi,z,t0))
    z_prod = hat_z(z,mu_bi,t0,0)**q0[0] * hat_z(z,mu_bi,t0,1)**q0[1]
    exp_component1 = np.exp(-l0[0]*s_vec[-1][0] - l_inf_bi[0]*alpha_bi[0]*integrate.simps(s_vec[:,0],t_vec_ODE))
    exp_component2 = np.exp(-l0[1]*s_vec[-1][1] - l_inf_bi[1]*alpha_bi[1]*integrate.simps(s_vec[:,1],t_vec_ODE))
    return z_prod*exp_component1*exp_component2


# First order derivatives for JT conditional on value at t0

# First order moment E[Q_1(t)|Q_1(t0) = q0]
def derivative_JT_bi_z1_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    return (JT(t,s,[z[0]+.5*h,z[1]],t0,l0,q0) - JT(t,s,[z[0]-.5*h,z[1]],t0,l0,q0))/h
# First order E[\lambda_1(t)|\lambda_1(t0) = \lambda_0]
def derivative_JT_bi_s1_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    return (JT(t,[s[0]+.5*h,s[1]],z,t0,l0,q0) - JT(t,[s[0]-.5*h,s[1]],z,t0,l0,q0))/h


# Variance - from first and second order derivatives for JT conditional on value at t0

# Second order reduced moment E[Q_1(t)^{[2]}| Q_1(t0) = q0]
def derivative2nd_JT_bi_z1z1_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    return (JT(t,s,[z[0]+h,z[1]],t0,l0,q0) - 2*JT(t, s, z,t0,l0,q0) +JT(t, s, [z[0]-h,z[1]],t0,l0,q0))/(h**2)

def var_Q1_derivative_bi_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    sec_fact_Q1 = derivative2nd_JT_bi_z1z1_cond(JT, t, s, z, t0, l0, q0)
    first_Q1 = derivative_JT_bi_z1_cond(JT, t, s, z, t0, l0, q0)
    return sec_fact_Q1 + first_Q1 - first_Q1**2

# Second order moment E[\lambda_1(t)^2| \lambda_1(t0) = \lambda_0]
def derivative2nd_JT_bi_s1s1_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    return (JT(t,[s[0]+h,s[1]],z,t0,l0,q0) - 2*JT(t, s, z,t0,l0,q0) +JT(t,[s[0]-h,s[1]], z,t0,l0,q0))/(h**2)

def var_L1_derivative_bi_cond(JT, t, s, z, t0, l0, q0, h = 10**(-3)):
    sec_L1 = derivative2nd_JT_bi_s1s1_cond(JT, t, s, z, t0, l0, q0)
    first_L1 = derivative_JT_bi_s1_cond(JT, t, s, z, t0, l0, q0)
    return sec_L1 - first_L1**2
                                                

In [ ]:
''' 
    Joint Transform bivariate setting
    at timepoints t and tau
'''



# Add function for ODE of tilde_r_j
def tilde_r_rm(r,v,a,b,mu,z,y,tau,j):
    drdv = a[j]*r[j] + (1+(y[j]*hat_z(z,mu,tau,j) -1)*np.exp(-mu[j]*v))*beta_expLT(b,r,j) - 1
    return -drdv

# Define the ODE for the system of ODEs for tilde_r
def tilde_r_system_rm(r,v,a,b,mu,z,y,tau):
    drdv = [tilde_r_rm(r,v,a,b,mu,z,y,tau,0), tilde_r_rm(r,v,a,b,mu,z,y,tau,1)]
    return drdv

def tilde_s_system_init(s,u,a,b,mu,z,t0):
    dsdu = [tilde_s_init(s,u,a,b,mu,z,0,t0), tilde_s_init(s,u,a,b,mu,z,1,t0)]
    return dsdu


def zeta_bivariate_tau_variant(t,tau,r,y,s,z):
    
    # Interval [t,t+tau] part
    t_tau_vec_ODE = np.linspace(t,t+tau,M)
    
    # Solution system of odes [tilde_s1, tilde_s2]
    s_tau_vec = integrate.odeint(tilde_s_system_init_bi_rm, s, t_tau_vec_ODE, args=(alpha_bi,B_bi,mu_bi,z,t))
    
    # Exponentials for components 1 and 2: integrated tilde_s
    interval_ttau1 = np.exp(-l_inf_bi[0]*alpha_bi[0]*integrate.simps(s_tau_vec[:,0],t_tau_vec_ODE))
    interval_ttau2 = np.exp(-l_inf_bi[1]*alpha_bi[1]*integrate.simps(s_tau_vec[:,1],t_tau_vec_ODE))
    
    # Interval [0,t] part
    t_vec_ODE = np.linspace(0,t,M)
    
     # Solution system of odes [tilde_r1, tilde_r2]
    r_vec = integrate.odeint(tilde_r_system_rm, r+s_tau_vec[-1], t_vec_ODE, args=(alpha_bi,B_bi,mu_bi,z,y,tau))
    
    # Exponentials for components 1 and 2: initial condition and integrated tilde_r
    interval_0t1 = np.exp(-l_inf_bi[0]*r_vec[-1][0] - l_inf_bi[0]*alpha_bi[0]*integrate.simps(r_vec[:,0],t_vec_ODE))
    interval_0t2 = np.exp(-l_inf_bi[1]*r_vec[-1][1] - l_inf_bi[1]*alpha_bi[1]*integrate.simps(r_vec[:,1],t_vec_ODE))
    
    return interval_ttau1*interval_ttau2*interval_0t1*interval_0t2




# Numerical derivative first order

# First order moment E[Q_1(t+\tau)]
def derivative_JT_bi_z1_tau(JT, t, tau, r, y, s, z, h = 10**(-5)):
    return (JT(t,tau, r, y, s, [z[0]+.5*h,z[1]]) - JT(t,tau, r, y, s, [z[0]-.5*h,z[1]]))/h
    
# First order moment E[Q_1(t)]
def derivative_JT_bi_y1_tau(JT, t, tau, r, y, s, z, h = 10**(-5)):
    return (JT(t,tau, r, [y[0]+.5*h,y[1]], s, z) - JT(t,tau, r, [y[0]-.5*h,y[1]], s, z))/h
    
# First order moment E[\lambda_1(t+\tau)]
def derivative_JT_bi_s1_tau(JT, t, tau, r, y, s, z, h = 10**(-5)):
    return -(JT(t,tau, r, y, [s[0]+.5*h,s[1]], z) - JT(t,tau, r, y, [s[0]-.5*h,s[1]], z))/h

# First order moment E[\lambda_1(t)]
def derivative_JT_bi_r1_tau(JT, t, tau, r, y, s, z, h = 10**(-5)):
    return -(JT(t,tau, [r[0]+.5*h,r[1]], y, s, z) - JT(t,tau, [r[0]-.5*h,r[1]], y, s, z))/h


# Numerical derivatives second order

# Second order mixed moment E[Q_1(t)*Q_1(t+\tau)]
def derivative2nd_JT_bi_y1z1_tau_cen(JT, t, tau, r, y, s, z, h = 10**(-3)):
    return (JT(t,tau, r, [y[0]+h,y[1]], s,[z[0]+h,z[1]]) - JT(t, tau, r, [y[0]+h,y[1]], s, [z[0]-h,z[1]]) - JT(t, tau, r, [y[0]-h,y[1]], s, [z[0]+h,z[1]]) + JT(t, tau, r, [y[0]-h,y[1]], s,[z[0]-h,z[1]]))/(4*h**2)

# Second order mixed moment E[Q_1(t)*Q_2(t+\tau)]
def derivative2nd_JT_bi_y1z2_tau_cen(JT, t, tau, r, y, s, z, h = 10**(-3)):
    return (JT(t,tau, r, [y[0]+h,y[1]], s,[z[0],z[1]+h]) - JT(t, tau, r, [y[0]+h,y[1]], s, [z[0],z[1]-h]) - JT(t, tau, r, [y[0]-h,y[1]], s, [z[0],z[1]+h]) + JT(t, tau, r, [y[0]-h,y[1]], s,[z[0],z[1]-h]))/(4*h**2)

# Second order mixed moment E[\lambda_1(t)*\lambda_1(t+\tau)]
def derivative2nd_JT_bi_r1s1_tau_cen(JT, t, tau, r, y, s, z, h = 10**(-3)):
    return (JT(t,tau, [r[0]+h,r[1]], y, [s[0]+h,s[1]],z) - JT(t, tau, [r[0]+h,r[1]], y, [s[0]-h,s[1]], z) - JT(t, tau, [r[0]-h,r[1]], y, [s[0]+h,s[1]], z) + JT(t, tau, [r[0]-h,r[1]], y, [s[0]-h,s[1]], z))/(4*h**2)

# Second order mixed moment E[\lambda_1(t)*\lambda_2(t+\tau)]
def derivative2nd_JT_bi_r1s2_tau_cen(JT, t, tau, r, y, s, z, h = 10**(-3)):
    return (JT(t,tau, [r[0]+h,r[1]], y, [s[0],s[1]+h],z) - JT(t, tau, [r[0]+h,r[1]], y, [s[0],s[1]-h], z) - JT(t, tau, [r[0]-h,r[1]], y, [s[0],s[1]+h], z) + JT(t, tau, [r[0]-h,r[1]], y, [s[0],s[1]-h], z))/(4*h**2)



